# Build a basic version of our model

## Import packages

In [ ]:
library(caret)
library(dplyr) #install.packages("tidyverse")
library(tidyr) 
library(ggplot2)
library(purrr)  
library(zoo) #install.packages("zoo")
library(parallel)
library(nimble) #install.packages("nimble")
library(abind) #install.packages("abind")
library(coda)
#library(caret) #install.packages("caret")
library(reticulate) #install.packages("reticulate")
library(data.table) #install.packages("data.table)
use_python("/usr/bin/python3")
#np = import("numpy") #py_install("numpy")
#pd = import("pandas") #py_install("pandas")

library(lcmm)

.libPaths('~/R_packages')

library(philentropy)
library(plgp)



setwd('~/northstar-trajectories/HierarchicalGaussianProcess/091023Run/HibmaModel')

set.seed(12345)



## Load in Data (and tidy)

In [ ]:
data = read.csv(file = 'in.csv')
#N_data_used = 1000
#data = data[0:N_data_used,]

N_patients_pre_split = length(unique(data$PatID))
N_valid = round(0.3*N_patients_pre_split) #30% of patients become the validaiton patients (data is pre-randomised)
validation_pats = unique(data$PatID)[1:N_valid]
training_pats = unique(data$PatID)[(N_valid+1):N_patients_pre_split]

data_validation = data[data$PatID %in% validation_pats,]
data = data[data$PatID %in% training_pats,]

#FIt model from scratch, or load in previously obtained model fits? 
#(The latter requires the data and inputs to not have been changed, but does allow new plots and predictions to be done)
Fit_from_scratch = FALSE
load_results <- FALSE
plot_preds <- TRUE
save_results <- TRUE

#which columns do we want in the model?
inputs =  c("steroids_regime")
outputs = c("Calculated_nsaa_score", "nsaa_walk_time", "nsaa_rise_from_floor_time")

#make data in terms of 3 month intervals
every_x_months = 3
data$fup_age_at_date_of_assessment = round(data$fup_age_at_date_of_assessment/every_x_months)

#Delete rows where the patients age aren't known (alternatively, we could re-infer it from the time?)
data = data[!is.na(data$fup_age_at_date_of_assessment), ]

#fill in missing timesteps (monthly)
data = complete(data, fup_age_at_date_of_assessment = 0:(25*12/every_x_months), nesting(PatID))

#also make sure we only have one data row per patID per appointment time:
data = data %>%
  group_by(PatID, fup_age_at_date_of_assessment) %>%
  filter(row_number()==1) %>%
  ungroup()

#inputs (i.e. treatments / X in the model) cannot have any missing values at any time step.
#This shouldn't be a problem as they dont need to be specially observed (clinicians should be dictating them)
#but they may not have been written down for each timestep

#sometimes data steroids is input as "" rather than NA
data$steroids_used[which(data$steroids_used == "")] = NA
data$steroids_regime[which(data$steroids_regime == "")] = NA

#for the treatments, which we don't expect to change regularly, use the previous value if there is one
data <- data %>% group_by(PatID) %>%
  fill(age_at_km_steroids_start, .direction = "downup") %>%
  fill(names(select(data, all_of(c(inputs, "steroids_dose")))), .direction = "down") %>%
  dplyr::ungroup()

#if the patient's age is after the age they started steroids, we can also use later values to interpolate
#Note that this will only do so if the date they started steroids is recorded (at any point, and is why its interpolated up and down earlier)
data <- data %>% group_by(PatID) %>%
  mutate(steroids_dose = ifelse(is.na(steroids_dose) & fup_age_at_date_of_assessment> age_at_km_steroids_start, na.locf(steroids_dose, fromLast = TRUE), steroids_dose)) %>%
  mutate(steroids_used = ifelse(is.na(steroids_used) & fup_age_at_date_of_assessment> age_at_km_steroids_start, na.locf(steroids_used, fromLast = TRUE, coredata = TRUE), steroids_used)) %>%
  mutate(steroids_regime = ifelse(is.na(steroids_regime) & fup_age_at_date_of_assessment> age_at_km_steroids_start, na.locf(steroids_regime, fromLast = TRUE, coredata = TRUE), steroids_regime)) %>%
  dplyr::ungroup()

#and if any steroids dose are still NA, make them 0
data <- data %>% mutate(steroids_dose = replace_na(steroids_dose, 0))
#and then, if any steroids dose are 0, force the other steroid information to be NA
data$steroids_used[data$steroids_dose == 0] = NA
data$steroids_regime[data$steroids_dose == 0] = NA

#for the other steroid information, replace NA with "Unknown" if steroids dose > 0, and "None" if steroids_dose = 0
which_missing_used = is.na(data$steroids_used) & (data$steroids_dose > 0)
which_none_used = is.na(data$steroids_used) & (data$steroids_dose == 0)
which_missing_regime = is.na(data$steroids_regime) & (data$steroids_dose > 0)
which_none_regime = is.na(data$steroids_regime) & (data$steroids_dose == 0)

data$steroids_used <- as.character(data$steroids_used)
data$steroids_used[which_none_used] = "None"
if(sum(which_missing_used) > 0){
  data$steroids_used[which_missing_used] = "Unknown"
}

data$steroids_regime <- as.character(data$steroids_regime)
data$steroids_regime[which_none_regime] = "None"
if(sum(which_missing_regime) > 0){
  data$steroids_regime[which_missing_regime] = "Unknown"
}


#convert other information to factors, and force "no steroids" to be the baseline intercept
data$steroids_used <- as.factor(data$steroids_used)
data$steroids_used <- relevel(data$steroids_used, "None") #make "None" first
data$steroids_regime <- as.factor(data$steroids_regime)
data$steroids_regime <- relevel(data$steroids_regime, "None") #make "None" first


#sometimes a walk time or rise time of 0 is recorded. This is impossible, so replace with NA

data$nsaa_walk_time[which(data$nsaa_walk_time == 0)] = NA
data$nsaa_rise_from_floor_time[which(data$nsaa_rise_from_floor_time == 0)] = NA

#convert dataframe into 3d arrays (time, patient, column)
N_patients = n_distinct(data$PatID)
N_timesteps = n_distinct(data$fup_age_at_date_of_assessment)

#which columns do we want (first set should be PatID and fup_age, second should be inputs, third should be outputs)
data <-select(data, all_of(c(c("PatID", "fup_age_at_date_of_assessment"), inputs, outputs)))

#change format of data to allow conversion
data = arrange(data, PatID, fup_age_at_date_of_assessment)
data$PatID <- as.integer(as.numeric(factor(data$PatID)))
data = data.frame(data)

#convert categorical variables to one hot variables ("None" is the base example)
dmy <- dummyVars(" ~ .", data = data, fullRank = TRUE)
data <- data.frame(predict(dmy, newdata = data))

N_cols = dim(select(data,contains(outputs)))[2] #number of outputs
N_covs = dim(select(data,contains(inputs)))[2] #number of inputs
Xnames = colnames(data)[2+(1:N_covs)] #names of covariates (for posterity)

#convert to 3d array
data <- data %>%
  nest(-PatID) %>%    # collapse other columns to list column of data frames
  mutate(data = map(data, ~as.matrix(.x[-1]))) %>%    # drop dates from nested data frames and coerce each to matrix
  pull(data) %>%    # extract matrix list
  invoke(abind::abind, ., along = 3) %>%    # abind in 3rd dimension
  `dimnames<-`(list(as.character(unique(data$fup_age_at_date_of_assessment)), names(data)[3:(3+N_cols+N_covs-1)], unique(data$PatID)))    # set dimnames properly

#swap index ordering to (PatID, timestep, column) (currently (timestep, column, PatID))
data <- aperm(data, c(3,1,2))

#make sure they have at least 1 NSAA score 
# might slightly bias, but we dont really have any proper information otherwise...
which_enough = which(apply(!is.na(data[,,N_covs+1]), 1, sum)>0)
data = data[which_enough, , ]

#shuffle patient order (randomise)
data = data[sample(dim(data)[1]),1:dim(data)[2], 1:dim(data)[3]]

#subset data for speed reasons
init_age = 3
final_age = 20
N_patients = dim(data)[1] #all of the patients

data = data[1:dim(data)[1], (init_age*12/every_x_months):(final_age*12/every_x_months), ]
#data = data[c(1,8), (init_age*12/every_x_months):(final_age*12/every_x_months), ]

#save full data 
X_data = data[,,1:N_covs,drop = FALSE]
y_data = data[,,(N_covs+1):(N_covs+N_cols),drop = FALSE]
save(X_data, y_data,file='y_data.out.RData')
#np$save("X_data.out.npy",r_to_py(X_data))
#np$save("y_data.out.npy",r_to_py(y_data))

#subset N patients as well
data = data[1:N_patients, , ]

N_timesteps = dim(data)[2]

sum(!is.na(data[,,N_covs+1])) #how many data points do we have for NSAA (the first output, use +2 for the second, etc)

#get input and output data
X_data = data[,,1:N_covs,drop = FALSE]
y_data = data[,,(N_covs+1):(N_covs+N_cols),drop = FALSE]

#get maximum number of timesteps observed for each patient, for each output
N_timesteps_t = matrix(1, nrow = N_patients, ncol = N_cols)
for (output in 1:N_cols){
  for (n in 1:N_patients){
    N_timesteps_t[n, output] = max(which(!is.na(y_data[n,,output])))
  }
}

#replace -Inf with 2 (minimum number of timestpes we'd need for model to actually run
N_timesteps_t[N_timesteps_t==-Inf] = 2

#Decide which patients/observations are the held-out validation points
#We need to hold back certain observations from them, and make the model predict for the whole timeseries for them

N_valid = round(0.2*N_patients) #20% of patients become the validaiton patients
#delete the second X% of observations for the first set of patients, where X% is random (between 20% and 80%)
y_gutted_data = y_data
y_valid = y_data






#transform the data
NSAA_transformer <- function(y){
  ytrans = y
  ytrans = logit( (ytrans + 0.5)/35) #convert -0.5 to 34.5 range to real range 
  return(ytrans)
}

positive_transformer <- function(y){
  ytrans = y
  ytrans = log(exp(ytrans)-1) #convert positive range to real range
    
  #make any values that are now inf what they were from y (its just a numerical issue caused by exp(big value)):
  ytrans[which(ytrans == Inf)] = y[which(ytrans == Inf)]
  return(ytrans)
}

NSAA_itransformer <- function(y){
  ytrans = y
  ytrans = ilogit(ytrans)*35 - 0.5  #convert real range to -0.5 to 34.5 range 
  return(ytrans)
}

positive_itransformer <- function(y){
  ytrans = y
  ytrans = log(1+exp(ytrans)) #convert real range to positive range
    
  #make any values that are now inf what they were from y (its just a numerical issue caused by exp(big value)):
  ytrans[which(ytrans == Inf)] = y[which(ytrans == Inf)]
  return(ytrans)
}

#y_gutted_data[,,1] = NSAA_transformer(y_gutted_data[,,1] )
#y_gutted_data[,,2] = positive_transformer(y_gutted_data[,,2] )
#y_gutted_data[,,3] = positive_transformer(y_gutted_data[,,3] )

X_max = apply(X_data, 3, max, na.rm = TRUE)
X_min = apply(X_data, 3, min, na.rm = TRUE)


X_scale = (X_max-X_min)
X_scale[X_scale == 0] = 1 #prevent NAs if only one value
X = sweep(sweep(X_data, 3, X_min, "-"), 3, X_scale, "/")
y_gutted = y_gutted_data


time_step_multi <- 12/every_x_months

#save standardisation values
#save(X_min, X_max, y_mean, y_sd,file='stand_vals.out.RData')
#np$save("X_min.out.npy",r_to_py(X_min))
#np$save("X_max.out.npy",r_to_py(X_max))
#np$save("y_mean.out.npy",r_to_py(y_mean))
#np$save("y_sd.out.npy",r_to_py(y_sd))



In [ ]:
solveLeastSquares <- nimbleFunction(
    run = function(X = double(2), y = double(1)) { # type declarations
        ans <- inverse(t(X) %*% X) %*% (t(X) %*% y)
        return(ans)
        returnType(double(2))  # return type declaration
    } )## Create Nimble Model (Code for the Bayesian Hierarchical Model)
#get Nimble Model
state_model_code <- nimbleCode({
  
    #init gradient
    #init_grad_mean ~ T(dnorm(0,sd=20),0,)
    #init_grad_sd ~ T(dnorm(0,sd=10),0,)
    #for (n in 1:N_patients){
    #    init_grad[n] ~ T(dnorm(init_grad_mean,sd=init_grad_sd),0,)
    #    grad[1,n] <- init_grad[n]
    #}
    
    #init nsaa
    #init gradient
    init_nsaa_mean ~ T(dnorm(0,sd=20),0,)
    init_nsaa_sd ~ T(dnorm(0,sd=10),0,)
    for (n in 1:N_patients){
        init_nsaa[n] ~ T(dnorm(init_nsaa_mean,sd=init_nsaa_sd),0,)
        nsaa[1,n] <- init_nsaa[n]
    }
  
    #theta_k_in
    theta_k_in_mean ~ T(dnorm(0,sd=20),0,)
    theta_k_in_sd ~ T(dnorm(0,sd=10),0,)
    for (n in 1:N_patients){
        theta_k_in[n] ~ T(dnorm(theta_k_in_mean,sd=theta_k_in_sd),0,)
    }
    
    #theta_k_out
    theta_k_out_mean ~ T(dnorm(0,sd=20),0,)
    theta_k_out_sd ~ T(dnorm(0,sd=10),0,)
    for (n in 1:N_patients){
        theta_k_out[n] ~ T(dnorm(theta_k_out_mean,sd=theta_k_out_sd),0,)
    }
    
    #chain
    for (time in 2:69){
        for (individual in 1:N_patients){
            #grad[time,individual] <- grad[time-1,individual] + theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
            #grad[time,individual] <- theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
            nsaa[time,individual] <- nsaa[time-1,individual] + theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
        }
    }
    
  
    #innovation  
    #for (output in 1:N_cols) {
      innov_mean ~ T(dnorm(0, sd = 5),0,)
      innov_sd ~ T(dnorm(0, sd = 5),0,)
      for (n in 1:N_patients) {        
        innov[n] ~ T(dnorm(innov_mean, sd = innov_sd),0,)
      }
    #}
                             
  #obs err
    #for (output in 1:N_cols) {
      obs_err_mean ~ T(dnorm(0, sd = 5),0,)
      obs_err_sd ~ T(dnorm(0, sd = 5),0,)
      for (n in 1:N_patients) {        
        obs_err[n] ~ T(dnorm(obs_err_mean, sd = obs_err_sd),0,)
      }
    #}
                             
  #Random walks
  #for (output in 1:N_cols) {
    for (n in 1:N_patients) {
      RW[n, 1] <- 0
      for (t in 2:N_timesteps_t_max[n]) {
        RW[n, t] ~ dnorm(RW[n,t-1], sd = innov[n])
      }
    }
  #}

  
  #observations:
  for (data in (1:N_data)) {
    y_state[data] <- nsaa[t_reshape[data],n_reshape[data]] + RW[n_reshape[data], t_reshape[data]]
    y[data] ~ dnorm( y_state[data], sd = obs_err[n_reshape[data]])
  }
    
  
})

### Get data ready for nimble

In [ ]:
#don't want to be modelling the *observations* at each time point, which nimble mgitht ry to do even for all the NAs
#so we flatten y into a 1d array only containing the actual observations
#but then we need several arrays which allow us to relearn which patient/timestep/output each eentry is:

y_gutted_reshape <- as.numeric(y_gutted)
n_matrix <- y_gutted
for (n in 1:dim(n_matrix)[1]){
  n_matrix[n,,] = n
}
n_reshape <- as.numeric(n_matrix)

t_matrix <- y_gutted
for (t in 1:dim(t_matrix)[2]){
  t_matrix[,t,] = t
}
t_reshape <- as.numeric(t_matrix)

out_matrix <- y_gutted
for (out in 1:dim(out_matrix)[3]){
  out_matrix[,,out] = out
}
out_reshape <- as.numeric(out_matrix)

y_gutted_reshape_isna = which(!is.na(y_gutted_reshape))
y_gutted_reshape = y_gutted_reshape[y_gutted_reshape_isna]
n_reshape = n_reshape[y_gutted_reshape_isna]
t_reshape = t_reshape[y_gutted_reshape_isna]
out_reshape = out_reshape[y_gutted_reshape_isna]

#For hibma mode, we only fit nsaa model, so remove all with outcome != 1
y_gutted_reshape_isna = which(!is.na(y_gutted_reshape))
y_gutted_reshape = y_gutted_reshape[out_reshape == 1]
n_reshape = n_reshape[out_reshape == 1]
t_reshape = t_reshape[out_reshape == 1]
out_reshape = out_reshape[out_reshape == 1]

#Due to castastrophic difficulties with indexing in nimble I have resorted to using these ad-hoc, hacky indexers to get it to work
#I've forgotten what most of them do to be honest but I managed to get it to work somehow
#Good luck to anyone in the future tying to figure out what is going on
#I've tried to annotate some of it to help out but idk
#Feel free to contact me (Victor Applebaum) if you have any questions but I will probaly have forgotten most of it by the time you're looking at this

n_data_ids <- rep(NA,N_patients)
for (n in 1:N_patients){
    n_data_ids[n] <- sum(n_reshape==n)
}

#Ok so some of the patients have had there results removed for some reason, and the whole thing crashes when this happened
#So we create a new number, N_pat_with_points, which is like N_patients but only those that have points
#We can then use i_n[n] for 1:N_pat_with_points to point to the nth patient that actually has points
n_pat_with_points <- c()
for (n in 1:N_patients){
    if (length(n_reshape[n_reshape==n])>0){
        n_pat_with_points <- append(n_pat_with_points,n)
    }
}
i_n <- rep(NA,length(n_pat_with_points))
for (i in 1:length(n_pat_with_points)){
    i_n[i] <- (1:N_patients)[n_pat_with_points[i]]
}
N_pat_with_points <- length(n_pat_with_points)

i <- nimMatrix(NA,N_patients,max(n_data_ids))#indexing matrix for patients i[n,i_ind]= position of i_ind'th point for nth patient
for (n in 1:N_patients){
    if (length((1:length(n_reshape))[n_reshape==n])>0){
        data_n <- c()
        for (data in 1:length(n_reshape)){
            if (n_reshape[data] == n){
                data_n <- append(data_n,data)
            }
        }
        for (i_ind in 1:length(data_n)){
            i[n,i_ind] <- data_n[i_ind]
        }
    }
}

j <- nimArray(NA,dim = c(N_patients,N_cols,max(n_data_ids)))#indexes both patient and outcome j[n,m,i_ind] position of i_ind'th point for nth patient's mth outcome
for (n in 1:N_patients){
    for (m in 1:N_cols){
        if (length((1:length(n_reshape))[n_reshape==n & out_reshape==m])>0){
            data_n <- c()
            for (data in 1:length(n_reshape)){
                if (n_reshape[data] == n & out_reshape[data] == m){
                    data_n <- append(data_n,data)
                }
            }
            for (i_ind in 1:length(data_n)){
                j[n,m,i_ind] <- data_n[i_ind]
            }
        }
    }
}
i_j <- nimMatrix(NA,N_patients,N_cols)#number of points avaliable for the patient n's mth outcome
for (n in 1:N_patients){
    for (m in 1:N_cols){
        i_j[n,m] <- sum(is.na(j[n,m,])==FALSE)
    }
}
y_real_reindexed=nimArray(NA,dim=c(N_patients,N_cols,length(y_gutted_reshape)))#rearranges y which i need for some reason
for (n in 1:N_pat_with_points){
    for (output in 1:N_cols){
      for (data in 1:i_j[i_n[n],output]){
          y_real_reindexed[i_n[n],output,data] <- y_gutted_reshape[j[i_n[n],output,data]]

      }
    }
}

i_out <- nimMatrix(NA,N_patients,N_cols)
i_out_num <- nimArray(NA,dim=c(N_patients))
for (n in 1:N_patients){
    counter <- 0
    for (output in 1:N_cols){
        if (i_j[n,output] > 1){
            counter <- counter + 1
            i_out[n,counter] <- output
        }
    }
    i_out_num[n] <- sum(!is.na(i_out[n,]))
}
i_out_1 <- nimMatrix(NA,N_patients,N_cols)
i_out_num_1 <- nimArray(NA,dim=c(N_patients))
for (n in 1:N_patients){
    counter <- 0
    for (output in 1:N_cols){
        if (i_j[n,output] == 1){
            counter <- counter + 1
            i_out_1[n,counter] <- output
        }
    }
    i_out_num_1[n] <- sum(!is.na(i_out_1[n,]))
}


starts_mat <- nimArray(NA,c(N_pat_with_points,N_cols))#At some point things have to go in a big matrix, we need these two to tell us the positions we should be working in
ends_mat <- nimArray(NA,c(N_pat_with_points,N_cols))
for (n in 1:N_pat_with_points){
    for (m in 1:N_cols){
        if (m == 1 & n == 1){
            starts <- c(1)
            ends <- c(i_j[i_n[n],m])
        }
        starts <- append(starts,ends[length(ends)]+1)
        ends <- append(ends,i_j[i_n[n],m]+starts[length(starts)]-1)
        ends_mat[n,m] <- ends[length(ends)-1]
        starts_mat[n,m] <- ends_mat[n,m]-i_j[i_n[n]]+1
    }
}

starts_mat <- nimArray(NA,c(N_pat_with_points,N_cols))#At some point things have to go in a big matrix, we need these two to tell us the positions we should be working in
ends_mat <- nimArray(NA,c(N_pat_with_points,N_cols))
rolling_sum <- 0
for (n in 1:N_pat_with_points){
    for (m in 1:N_cols){
        starts_mat[n,m] <- rolling_sum + 1
        ends_mat[n,m] <- starts_mat[n,m] + i_j[i_n[n],m]-1
        rolling_sum <- rolling_sum + i_j[i_n[n],m]
    }
}







In [ ]:
#Before fitting MCMC, also find some properties about the data set for synthetic section

col_i = 1 #we are interested in NSAA - first outcome

    #We find the distribution of start points
    start_points <- c()
    for (id in unique(n_reshape[out_reshape==1])){
        start_points <- append(start_points,min(t_reshape[out_reshape==1 & n_reshape==id]))
    }


    #We find the distribution of rates of appointment attendance in the real data
    rates <- c()
    for (id in unique(n_reshape[out_reshape==1 & n_reshape > N_valid])){
        if (length(n_reshape[out_reshape==1 & n_reshape==id]) > 0){
            rates <- append(rates,(max(t_reshape[out_reshape==1 & n_reshape==id])-min(t_reshape[out_reshape==1 & n_reshape==id]))/length(n_reshape[out_reshape==1 & n_reshape==id]-1))
        }
    }

    ##Finding the probabilities of a line stopping
    #First we find the locations of the stops for each individual
    end_location <- array(NA,dim=c(length(unique(n_reshape[n_reshape>N_valid])),2))
    for (individual in 1:length(unique(n_reshape[n_reshape>N_valid]))){
        end_location[individual,1] = max(t_reshape[n_reshape==unique(n_reshape[n_reshape>N_valid])[individual] & out_reshape==col_i])
        end_location[individual,2] = y_gutted_reshape[n_reshape==unique(n_reshape[n_reshape>N_valid])[individual] & out_reshape==col_i & t_reshape==end_location[individual,1]] 
    }
    #Create grid
    grid_height <- 6
    grid_width <- 6
    n_stops <- array(NA,dim=c(grid_height,grid_width))#Number of stops within a box
    stopping_grid <- array(NA,dim=c(grid_height,grid_width))#Fraction of point in each box which is a stop
    for (i in 1:grid_height){
        for (j in 1:grid_width){
            #Find box locations
            age_min <- (j-1)*70/grid_width
            age_max <- j*70/grid_width
            nsaa_min <- (i-1)*35/grid_height
            nsaa_max <- i*35/grid_height
            n_stops[i,j] <- length((1:length(unique(n_reshape)))[end_location[,1]>=age_min & end_location[,1]<age_max & end_location[,2]>=nsaa_min & end_location[,2]<nsaa_max])
            stopping_grid[i,j] <- n_stops[i,j] / length(n_reshape[out_reshape==col_i & t_reshape>=age_min & t_reshape<age_max & y_gutted_reshape>=nsaa_min & y_gutted_reshape<nsaa_max])
            if (is.na(stopping_grid[i,j])){stopping_grid[i,j]<-0}
        }
    }

#save treatments for later
X_train = X

### Prepare MCMC

In [ ]:
N_timesteps_t_max <- apply(N_timesteps_t,1, max)

state_model_consts <- list(N_covs = N_covs, #Number of covariances (something to do with treatments)?
                           N_cols = N_cols, #Number of outputs (3)
                           N_timesteps = N_timesteps, #Number of timesteps
                           N_timesteps_t = N_timesteps_t, 
                           N_timesteps_t_max = N_timesteps_t_max,
                           N_patients = N_patients, #Number of patients
                           X = X,#Something to do with treatments
                           N_data = length(y_gutted_reshape), #Number of data points
                           n_reshape = n_reshape,#Patient identifiers for each point
                           t_reshape = t_reshape,#Time for each point
                           out_reshape = out_reshape,#Identifier for output
                           timesteps_per_year = 12/every_x_months,
                           mature_age_steps = (20-init_age)*12/every_x_months,
                           milestone_age_steps = (15-init_age)*12/every_x_months,
                           n_data_ids = n_data_ids,#number of points for each patient
                           i = i, #indexing matrix for patients i[n,i_ind]= position of i_ind'th point for nth patient
                           n_pat_with_points = n_pat_with_points,
                           i_n = i_n,
                           N_pat_with_points =N_pat_with_points,
                           j = j, #indexes both patient and outcome j[n,m,i_ind] position of i_ind'th point for nth patient's mth outcome
                           i_j = i_j,
                           starts_mat = starts_mat,
                           ends_mat = ends_mat,
                           i_out = i_out,
                           i_out_num = i_out_num,
                           i_out_1 = i_out_1,
                           i_out_num_1 = i_out_num_1
                          )
state_model_data <-  list(y = y_gutted_reshape)#When Using RW
#state_model_data <-  list(y = y_real_reindexed)#When Using GP
seed = 12345

In [ ]:
#load init values from previous run
load('state_samples.out.RData')
#state_model_inits <- state_samples$chain1[1000,]

In [ ]:
state_model_inits <- list(init_nsaa = array(NA,N_patients),
                            theta_k_in = array(NA,N_patients),
                            theta_k_out = array(NA,N_patients),
                            obs_err = array(NA,N_patients),
                            innov = array(NA,N_patients),
                            RW = array(NA,c(N_patients,69))
                          )
                          



state_model_inits[['init_nsaa_mean']] = state_samples$chain1[1000,'init_nsaa_mean']
state_model_inits[['init_nsaa_sd']] = state_samples$chain1[1000,'init_nsaa_sd']
state_model_inits[['theta_k_in_mean']] = state_samples$chain1[1000,'theta_k_in_mean']
state_model_inits[['theta_k_in_sd']] = state_samples$chain1[1000,'theta_k_in_sd']
state_model_inits[['theta_k_out_mean']] = state_samples$chain1[1000,'theta_k_out_mean']
state_model_inits[['theta_k_out_sd']] = state_samples$chain1[1000,'theta_k_out_sd']
state_model_inits[['obs_err_mean']] = state_samples$chain1[1000,'obs_err_mean']
state_model_inits[['obs_err_sd']] = state_samples$chain1[1000,'obs_err_sd']
state_model_inits[['innov_mean']] = state_samples$chain1[1000,'innov_mean']
state_model_inits[['innov_sd']] = state_samples$chain1[1000,'innov_sd']
for (individual in 1:state_model_consts$N_patients){
    state_model_inits[['init_nsaa']][individual] <- state_samples$chain1[1000,'init_nsaa_mean']
    state_model_inits[['theta_k_in']][individual] <- state_samples$chain1[1000,'theta_k_in_mean']
    state_model_inits[['theta_k_out']][individual] <- state_samples$chain1[1000,'theta_k_out_mean']
    state_model_inits[['obs_err']][individual] <- state_samples$chain1[1000,'obs_err_mean']
    state_model_inits[['innov']][individual] <- state_samples$chain1[1000,'innov_mean']
    for (time in 1:69){
        state_model_inits[['RW']][individual,time] <- 0
    }
}

    

## Run MCMC, this is the cell which takes forever, and could be skipped and the results loaded in instead

It will be skipped if Fit_from_scratch is not TRUE

In [ ]:

if (Fit_from_scratch == TRUE){
      #nimble function for calculating the softmax probs for a vector of scores
      softplus_nimble <- nimbleFunction(
        run = function(x = double(0)){
          return(log(1+exp(x)))
          returnType(double(0))
        })
      csoftplus_nimble <- compileNimble(softplus_nimble)
    

      #nimble function required for getting LKJ correlation matrix
      uppertri_mult_diag <- nimbleFunction(
        run = function(mat = double(2), vec = double(1)) {
          returnType(double(2))
          p <- length(vec)
          out <- matrix(nrow = p, ncol = p, init = FALSE)
          for(i in 1:p)
            out[ , i] <- mat[ , i] * vec[i]
          return(out)
        })
      cuppertri_mult_diag <- compileNimble(uppertri_mult_diag)




monitors=c("nsaa","y_state",
                                                          "RW","innov","innov_mean","innov_sd",
                                                          "obs_err","obs_err_mean","obs_err_sd",
                                                          "theta_k_in","theta_k_in_mean","theta_k_in_sd",
                                                          "theta_k_out","theta_k_out_mean","theta_k_out_sd",
                                                          "init_nsaa","init_nsaa_mean","init_nsaa_sd"
                                               )



      state_samples <- nimbleMCMC(code = state_model_code,
                                  constants = state_model_consts,
                                  data = state_model_data,
                                  inits = state_model_inits,
                                  monitors=monitors,
                                  nburnin=1000000,
                                  niter = 2000000,
                                  thin = 1000,
                                  #setSeed=1,
                                  samplesAsCodaMCMC = TRUE,
                                  nchains = 2)


}

## Save or load the data (decide depending on if we ran the previous cell)

In [ ]:
#Save (or load) results

if (Fit_from_scratch == TRUE){
    save(state_samples,file='state_samples.out_RERUN.RData')
}else{
    load('state_samples.out_RERUN.RData')
}


## Traceplots

In [ ]:
plot(state_samples[,"init_nsaa_mean"])
plot(state_samples[,"init_nsaa_sd"])

In [ ]:
plot(state_samples[,"theta_k_in_mean"])
plot(state_samples[,"theta_k_in_sd"])
plot(state_samples[,"theta_k_in[2]"])

In [ ]:
plot(state_samples[,"theta_k_out_mean"])
plot(state_samples[,"theta_k_out_sd"])
plot(state_samples[,"theta_k_out[5]"])

In [ ]:
plot(state_samples[,"obs_err_mean"])
plot(state_samples[,"obs_err_sd"])
plot(state_samples[,"obs_err[1]"])

In [ ]:
plot(state_samples[,"y_state[10]"])

## Get results from model

In [ ]:
state_samples_merged <- data.frame(do.call('rbind',state_samples))

In [ ]:
#extract population parameters from fit
#and take logs of positive only parameters

transformed_state_samples_merged <- list()

transformed_state_samples_merged$init_nsaa_mean <- c(log(state_samples_merged$init_nsaa_mean))
transformed_state_samples_merged$init_nsaa_sd <- c(log(state_samples_merged$init_nsaa_sd))
transformed_state_samples_merged$theta_k_in_mean <- c(log(state_samples_merged$theta_k_in_mean))
transformed_state_samples_merged$theta_k_in_sd <- c(log(state_samples_merged$theta_k_in_sd))
transformed_state_samples_merged$theta_k_out_mean <- c(log(state_samples_merged$theta_k_out_mean))
transformed_state_samples_merged$theta_k_out_sd <- c(log(state_samples_merged$theta_k_out_sd))
transformed_state_samples_merged$innov_mean <- c(log(state_samples_merged$innov_mean))
transformed_state_samples_merged$innov_sd <- c(log(state_samples_merged$innov_sd))
transformed_state_samples_merged$obs_err_mean <- c(log(state_samples_merged$obs_err_mean))
transformed_state_samples_merged$obs_err_sd <- c(log(state_samples_merged$obs_err_sd))

order = c('init_nsaa_mean','init_nsaa_sd',
          'theta_k_in_mean','theta_k_in_sd',
          'theta_k_out_mean','theta_k_out_sd',
          'innov_mean','innov_sd',
          'obs_err_mean','obs_err_sd'
         )
transformed_state_samples_merged =  transformed_state_samples_merged[order]

#now approaximate using multivariate normal
pop_par_mean = colMeans(t(matrix(unlist(transformed_state_samples_merged), ncol = 2000, byrow = TRUE)))
pop_par_cov  = cov(t(matrix(unlist(transformed_state_samples_merged), ncol = 2000, byrow = TRUE)))

save(pop_par_mean,file='RERUN_pop_par_mean.RData')
save(pop_par_cov,file='RERUN_pop_par_cov.RData')


In [ ]:
#New Nimble model for fitting individuals given the known population parmeters
solveLeastSquares <- nimbleFunction(
    run = function(X = double(2), y = double(1)) { # type declarations
        ans <- inverse(t(X) %*% X) %*% (t(X) %*% y)
        return(ans)
        returnType(double(2))  # return type declaration
    } )## Create Nimble Model (Code for the Bayesian Hierarchical Model)
#get Nimble Model

state_model_code_individual <- nimbleCode({
  
    for (n in 1:N_patients){
        parameters[n,1:10] ~ dmnorm(pop_par_mean[1:10],cov = pop_par_cov[1:10,1:10])
        init_nsaa_mean[n] <- exp(parameters[n,1])
        init_nsaa_sd[n] <- exp(parameters[n,2])
        theta_k_in_mean[n] <- exp(parameters[n,3])
        theta_k_in_sd[n] <- exp(parameters[n,4])
        theta_k_out_mean[n] <- exp(parameters[n,5])
        theta_k_out_sd[n] <- exp(parameters[n,6])
        innov_mean[n] <- exp(parameters[n,7])
        innov_sd[n] <- exp(parameters[n,8])
        obs_err_mean[n] <- exp(parameters[n,9])
        obs_err_sd[n] <- exp(parameters[n,10])
    }
        
    #init nsaa
    #init gradient
    for (n in 1:N_patients){
        init_nsaa[n] ~ T(dnorm(init_nsaa_mean[n],sd=init_nsaa_sd[n]),0,)
        nsaa[1,n] <- init_nsaa[n]
    }
  
    #theta_k_in
    for (n in 1:N_patients){
        theta_k_in[n] ~ T(dnorm(theta_k_in_mean[n],sd=theta_k_in_sd[n]),0,)
    }
    
    #theta_k_out
    for (n in 1:N_patients){
        theta_k_out[n] ~ T(dnorm(theta_k_out_mean[n],sd=theta_k_out_sd[n]),0,)
    }
    
    #chain
    for (time in 2:69){
        for (individual in 1:N_patients){
            #grad[time,individual] <- grad[time-1,individual] + theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
            #grad[time,individual] <- theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
            nsaa[time,individual] <- nsaa[time-1,individual] + theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
        }
    }
    
  
    #innovation  
    #for (output in 1:N_cols) {
      for (n in 1:N_patients) {        
        innov[n] ~ T(dnorm(innov_mean[n], sd = innov_sd[n]),0,)
      }
    #}
                             
  #obs err
    #for (output in 1:N_cols) {
      for (n in 1:N_patients) {        
        obs_err[n] ~ T(dnorm(obs_err_mean[n], sd = obs_err_sd[n]),0,)
      }
    #}
                             
  #Random walks
  #for (output in 1:N_cols) {
    for (n in 1:N_patients) {
      RW[n, 1] <- 0
      for (t in 2:69) {
        RW[n, t] ~ dnorm(RW[n,t-1], sd = innov[n])
      }
    }
  #}

  
  #observations:
  for (data in (1:N_data)) {
    y_state[data] <- nsaa[t_reshape[data],n_reshape[data]] + RW[n_reshape[data], t_reshape[data]]
    y[data] ~ dnorm( y_state[data], sd = obs_err[n_reshape[data]])
  }

  
})


In [ ]:
##Now we repeat data processing with the validation dataset instead
data = data_validation

set.seed(12345)


#which columns do we want in the model?
inputs =  c("steroids_regime")
outputs = c("Calculated_nsaa_score", "nsaa_walk_time", "nsaa_rise_from_floor_time")

#make data in terms of 3 month intervals
every_x_months = 3
data$fup_age_at_date_of_assessment = round(data$fup_age_at_date_of_assessment/every_x_months)

#Delete rows where the patients age aren't known (alternatively, we could re-infer it from the time?)
data = data[!is.na(data$fup_age_at_date_of_assessment), ]

#fill in missing timesteps (monthly)
data = complete(data, fup_age_at_date_of_assessment = 0:(25*12/every_x_months), nesting(PatID))

#also make sure we only have one data row per patID per appointment time:
data = data %>%
  group_by(PatID, fup_age_at_date_of_assessment) %>%
  filter(row_number()==1) %>%
  ungroup()

#inputs (i.e. treatments / X in the model) cannot have any missing values at any time step.
#This shouldn't be a problem as they dont need to be specially observed (clinicians should be dictating them)
#but they may not have been written down for each timestep

#sometimes data steroids is input as "" rather than NA
data$steroids_used[which(data$steroids_used == "")] = NA
data$steroids_regime[which(data$steroids_regime == "")] = NA

#for the treatments, which we don't expect to change regularly, use the previous value if there is one
data <- data %>% group_by(PatID) %>%
  fill(age_at_km_steroids_start, .direction = "downup") %>%
  fill(names(select(data, all_of(c(inputs, "steroids_dose")))), .direction = "down") %>%
  dplyr::ungroup()

#if the patient's age is after the age they started steroids, we can also use later values to interpolate
#Note that this will only do so if the date they started steroids is recorded (at any point, and is why its interpolated up and down earlier)
data <- data %>% group_by(PatID) %>%
  mutate(steroids_dose = ifelse(is.na(steroids_dose) & fup_age_at_date_of_assessment> age_at_km_steroids_start, na.locf(steroids_dose, fromLast = TRUE), steroids_dose)) %>%
  mutate(steroids_used = ifelse(is.na(steroids_used) & fup_age_at_date_of_assessment> age_at_km_steroids_start, na.locf(steroids_used, fromLast = TRUE, coredata = TRUE), steroids_used)) %>%
  mutate(steroids_regime = ifelse(is.na(steroids_regime) & fup_age_at_date_of_assessment> age_at_km_steroids_start, na.locf(steroids_regime, fromLast = TRUE, coredata = TRUE), steroids_regime)) %>%
  dplyr::ungroup()

#and if any steroids dose are still NA, make them 0
data <- data %>% mutate(steroids_dose = replace_na(steroids_dose, 0))
#and then, if any steroids dose are 0, force the other steroid information to be NA
data$steroids_used[data$steroids_dose == 0] = NA
data$steroids_regime[data$steroids_dose == 0] = NA

#for the other steroid information, replace NA with "Unknown" if steroids dose > 0, and "None" if steroids_dose = 0
which_missing_used = is.na(data$steroids_used) & (data$steroids_dose > 0)
which_none_used = is.na(data$steroids_used) & (data$steroids_dose == 0)
which_missing_regime = is.na(data$steroids_regime) & (data$steroids_dose > 0)
which_none_regime = is.na(data$steroids_regime) & (data$steroids_dose == 0)

data$steroids_used <- as.character(data$steroids_used)
data$steroids_used[which_none_used] = "None"
if(sum(which_missing_used) > 0){
  data$steroids_used[which_missing_used] = "Unknown"
}

data$steroids_regime <- as.character(data$steroids_regime)
data$steroids_regime[which_none_regime] = "None"
if(sum(which_missing_regime) > 0){
  data$steroids_regime[which_missing_regime] = "Unknown"
}


#convert other information to factors, and force "no steroids" to be the baseline intercept
data$steroids_used <- as.factor(data$steroids_used)
data$steroids_used <- relevel(data$steroids_used, "None") #make "None" first
data$steroids_regime <- as.factor(data$steroids_regime)
data$steroids_regime <- relevel(data$steroids_regime, "None") #make "None" first


#sometimes a walk time or rise time of 0 is recorded. This is impossible, so replace with NA

data$nsaa_walk_time[which(data$nsaa_walk_time == 0)] = NA
data$nsaa_rise_from_floor_time[which(data$nsaa_rise_from_floor_time == 0)] = NA

#convert dataframe into 3d arrays (time, patient, column)
N_patients = n_distinct(data$PatID)
N_timesteps = n_distinct(data$fup_age_at_date_of_assessment)

#which columns do we want (first set should be PatID and fup_age, second should be inputs, third should be outputs)
data <-select(data, all_of(c(c("PatID", "fup_age_at_date_of_assessment"), inputs, outputs)))

#change format of data to allow conversion
data = arrange(data, PatID, fup_age_at_date_of_assessment)
data$PatID <- as.integer(as.numeric(factor(data$PatID)))
data = data.frame(data)

#convert categorical variables to one hot variables ("None" is the base example)
dmy <- dummyVars(" ~ .", data = data, fullRank = TRUE)
data <- data.frame(predict(dmy, newdata = data))

N_cols = dim(select(data,contains(outputs)))[2] #number of outputs
N_covs = dim(select(data,contains(inputs)))[2] #number of inputs
Xnames = colnames(data)[2+(1:N_covs)] #names of covariates (for posterity)

#convert to 3d array
data <- data %>%
  nest(-PatID) %>%    # collapse other columns to list column of data frames
  mutate(data = map(data, ~as.matrix(.x[-1]))) %>%    # drop dates from nested data frames and coerce each to matrix
  pull(data) %>%    # extract matrix list
  invoke(abind::abind, ., along = 3) %>%    # abind in 3rd dimension
  `dimnames<-`(list(as.character(unique(data$fup_age_at_date_of_assessment)), names(data)[3:(3+N_cols+N_covs-1)], unique(data$PatID)))    # set dimnames properly

#swap index ordering to (PatID, timestep, column) (currently (timestep, column, PatID))
data <- aperm(data, c(3,1,2))

#make sure they have at least 1 NSAA score 
# might slightly bias, but we dont really have any proper information otherwise...
which_enough = which(apply(!is.na(data[,,N_covs+1]), 1, sum)>0)
data = data[which_enough, , ]

#shuffle patient order (randomise)
data = data[sample(dim(data)[1]),1:dim(data)[2], 1:dim(data)[3]]

#subset data for speed reasons
init_age = 3
final_age = 20
N_patients = dim(data)[1] #all of the patients

data = data[1:dim(data)[1], (init_age*12/every_x_months):(final_age*12/every_x_months), ]
#data = data[c(1,8), (init_age*12/every_x_months):(final_age*12/every_x_months), ]

#save full data 
X_data = data[,,1:N_covs,drop = FALSE]
y_data = data[,,(N_covs+1):(N_covs+N_cols),drop = FALSE]
save(X_data, y_data,file='y_data.out.RData')
#np$save("X_data.out.npy",r_to_py(X_data))
#np$save("y_data.out.npy",r_to_py(y_data))

#subset N patients as well
data = data[1:N_patients, , ]

N_timesteps = dim(data)[2]

sum(!is.na(data[,,N_covs+1])) #how many data points do we have for NSAA (the first output, use +2 for the second, etc)

#get input and output data
X_data = data[,,1:N_covs,drop = FALSE]
y_data = data[,,(N_covs+1):(N_covs+N_cols),drop = FALSE]

#get maximum number of timesteps observed for each patient, for each output
N_timesteps_t = matrix(1, nrow = N_patients, ncol = N_cols)
for (output in 1:N_cols){
  for (n in 1:N_patients){
    N_timesteps_t[n, output] = max(which(!is.na(y_data[n,,output])))
  }
}

#replace -Inf with 2 (minimum number of timestpes we'd need for model to actually run
N_timesteps_t[N_timesteps_t==-Inf] = 2

#Decide which patients/observations are the held-out validation points
#We need to hold back certain observations from them, and make the model predict for the whole timeseries for them

N_valid = N_patients
#delete the second X% of observations for the first set of patients, where X% is random (between 20% and 80%)
y_gutted_data = y_data
y_valid = y_data
for (n in 1:N_valid){
  which_notna = which(!is.na(y_data[n,,1]))
  how_many_obs = length(which_notna)
  Xperc = runif(1, 0.2, 0.8)
  keep_up_to = which_notna[floor(how_many_obs*Xperc)]
  if (length(keep_up_to)==0){
    keep_up_to = 0
  }
  y_gutted_data[n, (keep_up_to+1):N_timesteps, 1:N_cols] = NA
  y_valid[n, 1:(keep_up_to), 1:N_cols] = NA
  N_timesteps_t[n, 1:N_cols] = N_timesteps
}



#y_gutted_data[,,1] = NSAA_transformer(y_gutted_data[,,1] )
#y_gutted_data[,,2] = positive_transformer(y_gutted_data[,,2] )
#y_gutted_data[,,3] = positive_transformer(y_gutted_data[,,3] )

#standardise data

#don't recalculate this - keeping same sd and means as were in training data
X_max = apply(X_data, 3, max, na.rm = TRUE)
X_min = apply(X_data, 3, min, na.rm = TRUE)


X_scale = (X_max-X_min)
X_scale[X_scale == 0] = 1 #prevent NAs if only one value
X = sweep(sweep(X_data, 3, X_min, "-"), 3, X_scale, "/")
y_gutted = y_gutted_data


time_step_multi <- 12/every_x_months


#don't want to be modelling the *observations* at each time point, which nimble mgitht ry to do even for all the NAs
#so we flatten y into a 1d array only containing the actual observations
#but then we need several arrays which allow us to relearn which patient/timestep/output each eentry is:

y_gutted_reshape <- as.numeric(y_gutted)
n_matrix <- y_gutted
for (n in 1:dim(n_matrix)[1]){
  n_matrix[n,,] = n
}
n_reshape <- as.numeric(n_matrix)

t_matrix <- y_gutted
for (t in 1:dim(t_matrix)[2]){
  t_matrix[,t,] = t
}
t_reshape <- as.numeric(t_matrix)

out_matrix <- y_gutted
for (out in 1:dim(out_matrix)[3]){
  out_matrix[,,out] = out
}
out_reshape <- as.numeric(out_matrix)

y_gutted_reshape_isna = which(!is.na(y_gutted_reshape))
y_gutted_reshape = y_gutted_reshape[y_gutted_reshape_isna]
n_reshape = n_reshape[y_gutted_reshape_isna]
t_reshape = t_reshape[y_gutted_reshape_isna]
out_reshape = out_reshape[y_gutted_reshape_isna]

#For hibma mode, we only fit nsaa model, so remove all with outcome != 1
y_gutted_reshape_isna = which(!is.na(y_gutted_reshape))
y_gutted_reshape = y_gutted_reshape[out_reshape == 1]
n_reshape = n_reshape[out_reshape == 1]
t_reshape = t_reshape[out_reshape == 1]
out_reshape = out_reshape[out_reshape == 1]

In [ ]:
#fit all validation individuals

    softplus_nimble <- nimbleFunction(
      run = function(x = double(0)){
        return(log(1+exp(x)))
        returnType(double(0))
      })

    state_model_consts <- list(N_covs = N_covs, #Number of covariances (something to do with treatments)?
                           N_cols = N_cols, #Number of outputs (3)
                           N_patients = N_patients, #Number of patients
                           X = X,#Something to do with treatments
                           N_data = length(y_gutted_reshape), #Number of data points
                           n_reshape = n_reshape,#Patient identifiers for each point
                           t_reshape = t_reshape,#Time for each point
                           out_reshape = out_reshape,#Identifier for output
                           timesteps_per_year = 12/every_x_months,
                           mature_age_steps = (20-init_age)*12/every_x_months,
                           milestone_age_steps = (15-init_age)*12/every_x_months,
                               pop_par_mean = pop_par_mean,
                               pop_par_cov = pop_par_cov
                              )
    state_model_data <-  list(y = c(y_gutted_reshape))
    Inits = list(parameters = array(pop_par_mean,c(N_patients,10)),
                 init_nsaa = array(exp(pop_par_mean[1]),N_patients),
                 theta_k_in = array(exp(pop_par_mean[3]),N_patients),
                 theta_k_out = array(exp(pop_par_mean[5]),N_patients),
                 innov = array(exp(pop_par_mean[7]),N_patients),
                 obs_err = array(exp(pop_par_mean[9]),N_patients)
                 )
                 

    Monitors = c('nsaa','y_state','theta_k_out','parameters','theta_k_in','innov','obs_err','RW','init_nsaa')

        samples <- nimbleMCMC(code = state_model_code_individual,
                   constants = state_model_consts,
                   data = state_model_data,
                   init = Inits,
                   monitors = Monitors,
                   niter = 2000000,
                   nburnin = 1000000,
                   thin = 1000,
                   nchains = 2,
                   samplesAsCodaMCMC = TRUE)

    save(samples,file = paste0('Samples','.RData'))


In [ ]:
plot(samples[,'theta_k_out[1]'])
plot(samples[,'theta_k_out[2]'])
plot(samples[,'innov[3]'])
plot(samples[,'obs_err[4]'])

In [ ]:
samples_merged <- data.frame(do.call('rbind',samples))

#### Northstar

In [ ]:
if (load_results == FALSE){

softplus_nimble <- nimbleFunction(
  run = function(x = double(0)){
    return(log(1+exp(x)))
    returnType(double(0))
  })


#plot nsaa trajectory for a patient
col_i = 1

#counters for checking test points in the intervals
test_points_in <- rep(0,99)
test_points_tot <- rep(0,99)

#observations for sharpnesses at sharpness_interval by number of points
sharpness_interval <- 0.95
sharpness_interval2 <- 0.7
num_points <- array(NA,N_valid)
for (individual in 1:N_valid){num_points[individual] = length((1:69)[!is.na(y_gutted_data[individual, , col_i])])}
sharpnesses <- array(NA,dim=c(N_valid,max(num_points)+1))
sharpnesses2 <- array(NA,dim=c(N_valid,max(num_points)+1))
sharpness_count <- array(0,dim=c(max(num_points))+1)


for (n in 1:N_valid){
   
    #extract needed parts from model
    N_timesteps_t_max = 69
    nsaa = as.matrix(select(select(samples_merged,contains(paste0("nsaa."))),ends_with(paste0("..",n, "."))))
    RW = as.matrix(select(samples_merged,contains(paste0("RW.",n, "."))))
    obs_err = as.matrix(select(samples_merged,contains(paste0("obs_err.",n, "."))))

    
    y_n = nsaa + RW
    y_n = y_n + rnorm(length(y_n), 0, obs_err) #add noise
    for (iter in 1:2000){
        for (time in 1:69){
            y_n[iter,time] = min(max(y_n[iter,time],0),34.0000001)
        }
    }

    q_025_n = apply(y_n[,1:69], 2, quantile, 0.025, na.rm = T)
    q_15_n = apply(y_n[,1:69], 2, quantile, 0.15, na.rm = T)
    q_5_n = apply(y_n[,1:69], 2, quantile, 0.5, na.rm = T)
    q_85_n = apply(y_n[,1:69], 2, quantile, 0.85, na.rm = T)
    q_975_n = apply(y_n[,1:69], 2, quantile, 0.975, na.rm = T)

    time <- init_age+(1:69)*(every_x_months/12)
     N_timesteps_t_max = apply(N_timesteps_t,1, max)
    
    
      #Plot
    if (plot_preds == TRUE){
  print(
    #With GP
    ggplot() +
      geom_ribbon(aes(x=init_age+(1:69)*(every_x_months/12),
                      ymin= q_15_n,
                      ymax= q_85_n),
                  alpha = 0.5,
                  xmin=0,
                  xmax=270,
                  fill = 'lightseagreen')+
      geom_ribbon(aes(x=init_age+(1:69)*(every_x_months/12),
                      ymin= q_85_n,
                      ymax= q_975_n),
                  alpha = 0.5,
                  xmin=0,
                  xmax=270,
                  fill = 'lightskyblue')+
      geom_ribbon(aes(x=init_age+(1:69)*(every_x_months/12),
                      ymin= q_025_n,
                      ymax= q_15_n),
                  alpha = 0.5,
                  xmin=0,
                  xmax=270,
                  fill = 'lightskyblue')+
      geom_line(aes(x=init_age+(1:N_timesteps_t_max[n])*(every_x_months/12), y = q_5_n), col='black',size=3) +#GP posterior mean
      geom_point(aes(x=init_age+(1:N_timesteps)*(every_x_months/12), y = y_data[n, , col_i]), size=4, col='red') +#Test points
      geom_point(aes(x=init_age+(1:N_timesteps)*(every_x_months/12), y = y_gutted_data[n, , col_i]), size=4, col='blue') +#Design points
      #ggtitle(paste("Patient", n ,"NSAA prediction"))+
      coord_cartesian(ylim = c(0,35), xlim = c(min(time),max(time)))+
      labs(x='Years since birth', y='NSAA score')+
      theme(text = element_text(size = 25,colour='#003D3C'))
  )
    ggsave(paste0("RERUN_NSAA/NSAA_pred", n, ".pdf"))

    }
          #Check number of test points inside each interval
      for (interval in 1:99){#for each interval
          upper <- apply(y_n[,1:N_timesteps_t_max[n]], 2, quantile, 1-(1-interval/100)/2, na.rm = T)
          lower <-  apply(y_n[,1:N_timesteps_t_max[n]], 2, quantile, (1-interval/100)/2, na.rm = T)
          for (point in (1:69)[!is.na(y_data[n, , col_i])]){
              
              if (y_data[n, point, col_i] < upper[point] & y_data[n, point, col_i] > lower[point] & !(point %in% (1:69)[!is.na(y_gutted_data[n, , col_i])])){
                  test_points_in[interval] <- test_points_in[interval] +1
              }
              
              if (!is.na(y_data[n, point, col_i]) & !(point %in% (1:69)[!is.na(y_gutted_data[n, , col_i])])){
                  test_points_tot[interval] <- test_points_tot[interval] +1
              }
          }
          #also evaluate sharpness
          if (interval/100 == sharpness_interval){
              sharpnesses[n,length((1:69)[!is.na(y_gutted_data[n, , col_i])])+1] <- mean(upper-lower)
              sharpness_count[length((1:69)[!is.na(y_gutted_data[n, , col_i])])+1] <- sharpness_count[length((1:69)[!is.na(y_gutted_data[n, , col_i])])+1] + 1
          }
          if (interval/100 == sharpness_interval2){
              sharpnesses2[n,length((1:69)[!is.na(y_gutted_data[n, , col_i])])+1] <- mean(upper-lower)
          }
      }
    }
}

In [ ]:
save_results <- TRUE

if (load_results == TRUE){
    load('NSAA_sharpnesses.RData')
    load('sharpness_count.RData')
    load('test_points_in.RData')
    load('test_points_tot.RData')
}


#plot quantile coverage
ggplot()+
    geom_line(aes(x=(1:99)/100, y = (1:99)/100), col='black',size=3)+
    geom_line(aes(x=(1:99)/100, y = test_points_in/test_points_tot), col='lightseagreen',size=3)+
    labs(x='Prediction Interval', y='Proportion Validation Points Contained')+
    theme(text = element_text(size = 25,colour='#003D3C'),legend.position="top")+
    scale_color_manual(breaks=c('Model', 'Target'),
                     values=c('Model'='lightseagreen', 'Target'='black'))#
    ggsave(paste0("AnalysisPlots/NSAAQuantileCoverage", ".pdf"))

#Determine sharpness

sharpness_means <- array(NA,dim(sharpnesses)[2])
for (no_points_s in 1:dim(sharpnesses)[2]){
    sharpness_means[no_points_s] <- mean(sharpnesses[,no_points_s],na.rm = TRUE)
}
sharpness_means2 <- array(NA,dim(sharpnesses2)[2])
for (no_points_s in 1:dim(sharpnesses2)[2]){
    sharpness_means2[no_points_s] <- mean(sharpnesses2[,no_points_s],na.rm = TRUE)
}

print(paste('The sum abs differences from desired quantile coverage is',sum(abs(test_points_in/test_points_tot - (1:99)/100))))

sharpness_grouped <- array(NA,4)
sharpness_grouped[1] <- sharpness_means[1] * sharpness_count[1] / sum(sharpness_count[1])
sharpness_grouped[2] <- sum(sharpness_means[2:4] * sharpness_count[2:4]) / sum(sharpness_count[2:4])
sharpness_grouped[3] <- sum(sharpness_means[5:8] * sharpness_count[5:8]) / sum(sharpness_count[5:8])
sharpness_grouped[4] <- sum(sharpness_means[9:length(sharpness_means)] * sharpness_count[9:length(sharpness_means)],na.rm=TRUE) / sum(sharpness_count[9:length(sharpness_means)],na.rm=TRUE)
print('Grouped together, the 95% mean prediction intervals are:')
sharpness_grouped
print('MCIDs:')
sharpness_grouped/3.5

sharpness2_grouped <- array(NA,4)
sharpness2_grouped[1] <- sharpness_means2[1] * sharpness_count[1] / sum(sharpness_count[1])
sharpness2_grouped[2] <- sum(sharpness_means2[2:4] * sharpness_count[2:4]) / sum(sharpness_count[2:4])
sharpness2_grouped[3] <- sum(sharpness_means2[5:8] * sharpness_count[5:8]) / sum(sharpness_count[5:8])
sharpness2_grouped[4] <- sum(sharpness_means2[9:length(sharpness_means2)] * sharpness_count[9:length(sharpness_means2)],na.rm=TRUE) / sum(sharpness_count[9:length(sharpness_means2)],na.rm=TRUE)
print('Grouped together, the 70% mean prediction intervals are:')
sharpness2_grouped
print('MCIDs:')
sharpness2_grouped/3.5

if (save_results == TRUE){
    save(sharpnesses,file='NSAA_sharpnesses.RData')
    save(sharpness_count,file='sharpness_count.RData')
    save(test_points_in,file='test_points_in.RData')
    save(test_points_tot,file='test_points_tot.RData')
}


## Could also use this model for synthetic data generation

In [ ]:
Generate_Trajectory <- function(){
    N_covs = N_covs #Number of covariances (something to do with treatments)?
                               N_cols = N_cols #Number of outputs (3)
                               N_timesteps = 69 #Number of timesteps
                               N_timesteps_t = 69 
                               N_timesteps_t_max = 69
                               N_patients = 1 #Number of patients
                               X = X_train[sample(dim(X_train)[1],1),,]#Something to do with treatments
                               N_data = length(y_gutted_reshape[n_reshape == individual]) #Number of data points
                               n_reshape = rep(1,length(n_reshape[n_reshape == individual]))#Patient identifiers for each point
                               t_reshape = as.array(t_reshape[n_reshape == individual])#Time for each point
                               out_reshape = as.array(out_reshape[n_reshape == individual])#Identifier for output
                               timesteps_per_year = 12/every_x_months
                               mature_age_steps = (20-init_age)*12/every_x_months
                               milestone_age_steps = (15-init_age)*12/every_x_months
    
    parameters <- array(NA,10)
    parameters[1:10] <- rmvnorm(1,pop_par_mean[1:10],pop_par_cov[1:10,1:10])
    
        init_nsaa_mean <- exp(parameters[1])
        init_nsaa_sd <- exp(parameters[2])
        theta_k_in_mean <- exp(parameters[3])
        theta_k_in_sd <- exp(parameters[4])
        theta_k_out_mean <- exp(parameters[5])
        theta_k_out_sd <- exp(parameters[6])
        innov_mean <- exp(parameters[7])
        innov_sd <- exp(parameters[8])
        obs_err_mean <- exp(parameters[9])
        obs_err_sd <- exp(parameters[10])

        
    #init nsaa
    #init gradient
    nsaa <- array(NA,69)
        init_nsaa <- max(rnorm(1,init_nsaa_mean,sd=init_nsaa_sd),0)
        nsaa[1] <- init_nsaa

  
    #theta_k_in
        theta_k_in <- max(rnorm(1,theta_k_in_mean,sd=theta_k_in_sd),0)
 
    
    #theta_k_out

        theta_k_out <- max(rnorm(1,theta_k_out_mean,sd=theta_k_out_sd),0)
 
    
    #chain

    for (time in 2:69){
            #grad[time,individual] <- grad[time-1,individual] + theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
            #grad[time,individual] <- theta_k_in[individual] - (theta_k_out[individual] * time) * nsaa[time-1,individual]
            nsaa[time] <- nsaa[time-1] + theta_k_in - (theta_k_out * time) * nsaa[time-1]

    }
    
  
    #innovation  
    #for (output in 1:N_cols) {    
        innov <- max(rnorm(1,innov_mean, sd = innov_sd),0)

    #}
                             
  #obs err
    #for (output in 1:N_cols) {       
        obs_err <- max(rnorm(1,obs_err_mean, sd = obs_err_sd),0)
    #}
                             
  #Random walks
  #for (output in 1:N_cols) {
    RW <- array(NA,69)
      RW[1] <- 0
      for (t in 2:69) {
        RW[t] <- rnorm(1,RW[t-1], sd = innov)
      }
  #}

  
  y_state <- array(NA,c(69))
  for (t in 1:69){
      y_state[t] <- nsaa[t] + RW[t] + rnorm(1,0,sd=obs_err)
  }
  for (t in 1:69){
      y_state[t] <- max(0,min(34,y_state[t]))
  }

  return(y_state)


}

In [ ]:
library(MASS)

#extracting trajectories

    set.seed(10)
KL_score <- array(NA,100)
LC_score <- array(NA,100)
for (synth_run in 1:100){    while(TRUE){ df <- try({
        N_synth_trajs <- N_valid
        N_show <- 50 #number of trajectories shown in the plots

        Synth_trajs <- array(NA,c(N_synth_trajs,69))
        for (synth_traj in 1:N_synth_trajs){
            Synth_trajs[synth_traj,] <- Generate_Trajectory()
        }


        #sample initial points
        synth_start_time <- array(NA,N_synth_trajs)
        for (i in 1:N_synth_trajs){
            synth_start_time[i] <- sample(start_points,1)
        }
        #sampling rates
        synth_rate <- array(NA,N_synth_trajs)
        for (i in 1:N_synth_trajs){
            synth_rate[i] <- round(sample(rates,1))
        }
        #if it is already at zero, trajectory doesn't exist, so resample
        for (i in 1:N_synth_trajs){
            no_zeroes <- FALSE
            while (no_zeroes == FALSE){
                if (Synth_trajs[i,synth_start_time[i]] == 0 | Synth_trajs[i,synth_start_time[i] + synth_rate[i]] == 0){
                    Synth_trajs[i,] <- round(Generate_Trajectory())
                } else {
                    no_zeroes <- TRUE
                }
            }
        }

        Synth_trajs_reshape <- reshape2::melt(Synth_trajs,varnames=c('individual','time'))
        Synth_trajs_reshape_limited <- Synth_trajs_reshape[Synth_trajs_reshape[,1] %in% 1:N_show,]
        if (synth_run == 1){
            synth_plot <- ggplot(Synth_trajs_reshape_limited) +
                geom_line(aes(x=(init_age+(1:N_timesteps)*(every_x_months/12))[time],y=value,group=individual,colour=factor(individual))) +
                #theme_bw() +
                labs(x='Years since birth', y='NSAA score')+
                theme(text = element_text(size = 25,colour='#003D3C'),legend.position="none") +
                xlim(3.25,20.25) +
                ylim(0,35)
            synth_plot
                ggsave("SynthTrajectoriesExampleStep1.pdf")
            }

        #now set initial points
        for (i in 1:N_synth_trajs){
            Synth_trajs_reshape <- Synth_trajs_reshape[!(Synth_trajs_reshape$individual == i & Synth_trajs_reshape$time < synth_start_time[i]),]
        }
        Synth_trajs_reshape_limited <- Synth_trajs_reshape[Synth_trajs_reshape[,1] %in% 1:N_show,]
        if (synth_run == 1){
            synth_plot <- ggplot(Synth_trajs_reshape_limited) +
                geom_line(aes(x=(init_age+(1:N_timesteps)*(every_x_months/12))[time],y=value,group=individual,colour=factor(individual))) +
                #theme_bw() +
                labs(x='Years since birth', y='NSAA score')+
                theme(text = element_text(size = 25,colour='#003D3C'),legend.position="none") +
                xlim(3.25,20.25) +
                ylim(0,35)
            synth_plot
                ggsave("SynthTrajectoriesExampleStep2.pdf")
        }

        #now do regularities
        for (i in 1:N_synth_trajs){
            Synth_trajs_reshape <- Synth_trajs_reshape[!(Synth_trajs_reshape$individual == i & Synth_trajs_reshape$time %in% (synth_start_time[i]+synth_rate[i]*(1:69))),]
        }
        Synth_trajs_reshape_limited <- Synth_trajs_reshape[Synth_trajs_reshape[,1] %in% 1:N_show,]
        if (synth_run == 1){
            synth_plot <- ggplot(Synth_trajs_reshape_limited) +
                geom_line(aes(x=(init_age+(1:N_timesteps)*(every_x_months/12))[time],y=value,group=individual,colour=factor(individual))) +
                #theme_bw() +
                labs(x='Years since birth', y='NSAA score')+
                theme(text = element_text(size = 25,colour='#003D3C'),legend.position="none") +
                xlim(3.25,20.25) +
                ylim(0,35)
            synth_plot
                ggsave("SynthTrajectoriesExampleStep3.pdf")
            }
    


        #stopping points
        last_point <- array(FALSE,dim(Synth_trajs_reshape)[1])
            for (i in 1:grid_height){
                for (j in 1:grid_width){
                    #Find box locations
                    age_min <- (j-1)*70/grid_width
                    age_max <- j*70/grid_width
                    nsaa_min <- (i-1)*35/grid_height
                    nsaa_max <- i*35/grid_height
                    for (point in 1:dim(Synth_trajs_reshape)[1]){
                        if (Synth_trajs_reshape$time[point] <= age_max & Synth_trajs_reshape$time[point] > age_min & Synth_trajs_reshape$value[point] <= nsaa_max & Synth_trajs_reshape$value[point] > nsaa_min){
                            last_point[point] <- sample(c(TRUE,FALSE),1,prob=c(stopping_grid[i,j],1-stopping_grid[i,j]))
                        }
                    }
                }
            }
        #remove points after these points
        remove_points <- c()
        for (point in 1:dim(Synth_trajs_reshape)[1]){
            if (last_point[point] == TRUE){
                for (point2 in 1:dim(Synth_trajs_reshape)[1]){
                    if (Synth_trajs_reshape$individual[point2] == Synth_trajs_reshape$individual[point] & Synth_trajs_reshape$time[point2] > Synth_trajs_reshape$time[point]){
                        remove_points <- append(remove_points,point2)
                    }
                }
            }
            if (Synth_trajs_reshape$value[point] == 0){
                for (point2 in 1:dim(Synth_trajs_reshape)[1]){
                    if (Synth_trajs_reshape$individual[point2] == Synth_trajs_reshape$individual[point] & Synth_trajs_reshape$time[point2] >= Synth_trajs_reshape$time[point]){
                        remove_points <- append(remove_points,point2)
                    }
                }
            }
        }

        Synth_trajs_reshape <- Synth_trajs_reshape[! (1:dim(Synth_trajs_reshape)[1] %in% remove_points),]
        Synth_trajs_reshape_limited <- Synth_trajs_reshape[Synth_trajs_reshape[,1] %in% 1:N_show,]
            if (synth_run == 1){
        synth_plot <- ggplot(Synth_trajs_reshape_limited) +
            geom_line(aes(x=(init_age+(1:N_timesteps)*(every_x_months/12))[time],y=value,group=individual,colour=factor(individual))) +
            #theme_bw() +
            labs(x='Years since birth', y='NSAA score')+
            theme(text = element_text(size = 25,colour='#003D3C'),legend.position="none") +
            xlim(3.25,20.25) +
            ylim(0,35)
        synth_plot
            ggsave("SynthTrajectoriesExampleStep4.pdf")
                }

        #combine fake and real trajectories
        y_data_reshape <- reshape2::melt(unname(y_data[,,1]),varnames=c('id','age'))[!is.na(reshape2::melt(unname(y_data[,,1]),varnames=c('id','age'))$value),]
        y_data_reshape$real <- TRUE
        y_test_real_reshape <- reshape2::melt(unname(y_gutted_data[, , 1]),varnames=c('id','age'))[!is.na(reshape2::melt(unname(y_gutted_data[, , 1]),varnames=c('id','age'))$value),]
        y_test_real_reshape$real <- TRUE
        Synth_trajs_reshape$real <- FALSE
        names(Synth_trajs_reshape) = c('id','age','value','real')
        Synth_trajs_reshape$id <- Synth_trajs_reshape$id + max(y_test_real_reshape$id,y_data_reshape$id)

        Trajs_together <- rbind(y_data_reshape,y_test_real_reshape,Synth_trajs_reshape)

        ng = 4

        # Run latent class model (takes time)
        LatC8 <- lcmm(value ~ age + I(age^2), mixture=~age, maxiter = 1000, idiag = TRUE, subject='id', ng=ng, data=Trajs_together)

       pred_ages = seq(1, max(Trajs_together$age,na.rm = TRUE),length=max(Trajs_together$age,na.rm = TRUE))
        dn <- data.frame(age=pred_ages)
        lcT8 <- predictY(LatC8, newdata = dn, var.time = "age", draws = TRUE)
        lcT8$age <- dn$age

        classes <- LatC8$pprob$class
        patNum <- LatC8$pprob$id

        tmp <- do.call(rbind, Map(data.frame, A=classes, B=patNum))

        lcT <- lcT8
        times = lcT$times[,1]
        pred = lcT$pred
        pred = data.frame(pred)

        group.colors <- c("Class 1" = "red", "Class 2" = "yellow", "Class 3" ="blue", "Class 4" = "darkgreen")

        i = 1
        for (x in tmp[["B"]]){
            Trajs_together[Trajs_together$id == x, "labels"] = tmp[["A"]][i]
            i = i +1
        }

        #need to remove patIDNum to maintain consistency with kmeans output
        #Trajs_together = select(Trajs_together, -id)

        #and make classes 0-3 rather than 1-4, again for kmeans consistency
        Trajs_together["labels"] = Trajs_together["labels"] -1 

        data <- Trajs_together[which(!is.na(Trajs_together$labels)),] #remove na classes (why do these exist?)

        data$labels <- as.factor(data$labels)
        levels(data$labels) <- c("Class 4", "Class 3", "Class 2", "Class 1")


        data.real <- data[data[,4] == TRUE,]
        data.synth <- data[data[,4] == FALSE,]

        #For first run only, we plot synthetic and real data
            if (synth_run == 1){
        print(
        ggplot(data)+
            geom_line(aes(x=init_age+age*(every_x_months/12), y= value, group = id, color = labels))+
            xlab("age")+
            scale_color_manual(values=group.colors)+
            theme(legend.position = "none",
            axis.text=element_text(size=16),
            axis.title=element_text(size=16))+
            ylab("NSAA")+
            xlab("Age (Years)")+
            coord_cartesian(xlim=c(3,20.25),ylim=c(0,35)))
            #ggtitle('Groupings of both real and synthetic data together'))
            #coord_cartesian(xlim=c(0, 69),ylim=c(-3,3))+
            #scale_y_continuous(breaks=seq(0,35,5), limits = c(0, 35))
        ggsave("NSAA_prediction_plots/data_all_clustered.out.png",width = 16, height = 14, units = "cm")

        print(
        ggplot(data.real)+
            geom_line(aes(x=init_age+age*(every_x_months/12), y= value, group = id, color = labels))+
            xlab("age")+
            scale_color_manual(values=group.colors)+
            theme(legend.position = "none",
            axis.text=element_text(size=16),
            axis.title=element_text(size=16))+
            ylab("NSAA")+
            xlab("Age (Years)")+
            coord_cartesian(xlim=c(3,20.25),ylim=c(0,35)))
            #ggtitle('Groupings of real data'))
            #coord_cartesian(xlim=c(0, 69),ylim=c(-3,3))+
            #scale_y_continuous(breaks=seq(0,35,5), limits = c(0, 35))
        ggsave("NSAA_prediction_plots/data_real_clustered.out.png",width = 16, height = 14, units = "cm")

        print(
        ggplot(data.synth)+
            geom_line(aes(x=init_age+age*(every_x_months/12), y= value, group = id, color = labels))+
            xlab("age")+
            scale_color_manual(values=group.colors)+
            theme(legend.position = "none",
            axis.text=element_text(size=16),
            axis.title=element_text(size=16))+
            ylab("NSAA")+
            xlab("Age (Years)")+
            coord_cartesian(xlim=c(3,20.25),ylim=c(0,35)))
            #ggtitle('Groupings of synthetic data')
            #coord_cartesian(xlim=c(0, 69),ylim=c(-3,3))+
            #scale_y_continuous(breaks=seq(0,35,5), limits = c(0, 35))
               ggsave("NSAA_prediction_plots/data_synth_clustered.out.png",width = 16, height = 14, units = "cm")
    }

        classifications <- data.frame(patNum,classes)
        classifications.real <- classifications[patNum <= max(n_reshape[out_reshape==1]),]
        classifications.synth <- classifications[patNum > max(n_reshape[out_reshape==1]),]

        g.real <- array(NA,4)
        g.synth <- array(NA,4)
        for (group in 1:ng){
            g.real[group] <- length(rep(1:dim(classifications.real)[1])[classifications.real$classes==group])
            g.synth[group] <- length(rep(1:dim(classifications.synth)[1])[classifications.synth$classes==group])
        }

        LC_score[synth_run] <- log((1/ng) * sum((g.real/(g.real+g.synth) - sum(g.real)/(sum(g.real)+sum(g.synth)))^2))



        esti1 <- kde2d(data.real$age,data.real$value,lims=c(0,69,0,35))
        esti1df <- expand.grid(X=esti1$x, Y=esti1$y)
        esti1df$Z <- c(esti1$z)

        esti2 <- kde2d(data.synth$age,data.synth$value,lims=c(0,69,0,35))
        esti2df <- expand.grid(X=esti2$x, Y=esti2$y)
        esti2df$Z <- c(esti2$z)


        KL_score[synth_run] <- KL(rbind(c(esti1$z)/sum(c(esti1$z)),c(esti2$z)/sum(c(esti2$z))))

    })
    if(!is(df, 'try-error')) break }
}

In [ ]:
mean(KL_score)
mean(LC_score)